# TIFF to MP4/GIF Conversion

In [ ]:
import napari
import tifffile as tiff
import os
import numpy as np
import dask.array as da
import dask
from dask.delayed import delayed
from napari_animation import Animation
import subprocess

# Set Dask to use multi-threading for better performance
dask.config.set(scheduler="threads")

import subprocess

def speed_up_video_ffmpeg(input_path, output_path, speed_factor):
    """
    Speeds up an MP4 video using FFmpeg with NVIDIA GPU hardware acceleration (NVENC),
    without processing the audio.
    
    :param input_path: Path to the input MP4 file.
    :param output_path: Path to save the output MP4 file.
    :param speed_factor: Factor by which to speed up the video (e.g., 2.0 for double speed).
    """
    # Adjust the setpts filter for speeding up the video
    video_filter = f"setpts={1/speed_factor}*PTS"
    
    # Create the FFmpeg command
    speed_cmd = [
    "ffmpeg", "-hwaccel", "cuda", "-i", input_path, 
    "-filter_complex", f"[0:v]{video_filter}[v]", 
    "-map", "[v]", 
    "-c:v", "h264_nvenc", "-preset", "veryfast",  
    "-vsync", "0",  
    "-threads", "8",  
    output_path
]

    
    try:
        # Execute the FFmpeg command and capture output
        result = subprocess.run(speed_cmd, check=True, capture_output=True, text=True)
        print(result.stdout)  # Print FFmpeg output
    except subprocess.CalledProcessError as e:
        # Print error output
        print("Error occurred:")
        print(e.stderr)




def convert_tiff_to_video(tiff_path: str, img_name: list[str] = None, file_format: str = 'mp4', fps: int = 30, speed_factor: float = 1.0):
    """
    Converts a folder of TIFF images into a video animation (MP4 or GIF) using Napari.
    
    Parameters:
        tiff_path (str): Base path to the folder containing TIFF images.
        img_name (list of str, optional): List of folder names to process. Defaults to None (processes all).
        file_format (str): Format of the output video ('mp4' or 'gif'). Defaults to 'mp4'.
        fps (int): Frames per second for the output video. Defaults to 30.
        speed_factor (float): Factor to speed up video by. Defaults to 1.0.
    """
    video_path = tiff_path.replace('tiff', 'video')
    os.makedirs(video_path, exist_ok=True)

    if img_name is None:
        img_name = [name for name in os.listdir(tiff_path) if os.path.isdir(os.path.join(tiff_path, name))]
    
    for name in img_name:
        folder_path = os.path.join(tiff_path, name)
        save_path = os.path.join(video_path, name)
        os.makedirs(save_path, exist_ok=True)

        # Get sorted list of TIFF files
        tiff_files = sorted(
            [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith('.tiff')]
        )

        if not tiff_files:
            print(f"Warning: No TIFF files found in {folder_path}. Skipping...")
            continue

        # Load images into a Dask array (parallel lazy loading)
        lazy_images = [delayed(tiff.imread)(f) for f in tiff_files]
        image_stack = da.stack([da.from_delayed(img, shape=(512, 512), dtype=np.uint8) for img in lazy_images])  # Adjust shape/dtype as needed

        # Open Napari viewer
        viewer = napari.Viewer()
        layer = viewer.add_image(image_stack, colormap='gray', name="TIFF Stack", scale=[1, 1, 1])

        # Create an animation
        animation = Animation(viewer)
        for i in range(image_stack.shape[0]):
            viewer.dims.set_point(0, i)  # Move through the Z-stack
            animation.capture_keyframe()

        # Save the animation
        output_file = os.path.join(save_path, f"{name}.{file_format}")
        animation.animate(output_file, fps=fps, file_format=file_format)  # Adjust fps dynamically

        print(f"Animation saved at: {output_file}")

        napari.run()

        if file_format=='mp4' and speed_factor!=1.0:
            speed_up_video_ffmpeg(output_file, output_file, speed_factor)

In [ ]:
tiff_path = r'D:\Darren\Files\outputs\tiff\pretrained_mic50_tab30_spray60'

convert_tiff_to_video(tiff_path, ['1_Microsphere'], 'mp4', 10.0)
convert_tiff_to_video(tiff_path, ['3_SprayDriedDispersion'], 'mp4', 10.0)

In [2]:
vid_path = r'd:\Darren\Files\outputs\video\pretrained_mic50_tab30_spray60\3_SprayDriedDispersion\3_SprayDriedDispersion.mp4'
speed_up_video_ffmpeg(vid_path, vid_path, 10.0)

Error occurred:
ffmpeg version 4.3.1 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 10.2.1 (GCC) 20200726
  configuration: --disable-static --enable-shared --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libsrt --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libgsm --enable-librav1e --disable-w32threads --enable-libmfx --enable-ffnvcodec --enable-cuda-llvm --enable-cuvid -